In [1]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
# To install this module, run:
# python -m pip install Pillow
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person

In [2]:
API_KEY = ''
ENDPOINT = ''
img = ''
face_client = FaceClient(ENDPOINT,CognitiveServicesCredentials(API_KEY))

# Face APIの仕様について
こちらのリンクを参照
https://docs.microsoft.com/en-us/python/api/overview/azure/cognitiveservices/faceapi?view=azure-python

In [3]:
# Used in the Person Group Operations and Delete Person Group examples.
# You can call list_person_groups to print a list of preexisting PersonGroups.
# SOURCE_PERSON_GROUP_ID should be all lowercase and alphanumeric. For example, 'mygroupname' (dashes are OK).
PERSON_GROUP_ID = str(uuid.uuid4()) # assign a random ID (or name it anything)

# Used for the Delete Person Group example.
TARGET_PERSON_GROUP_ID = str(uuid.uuid4()) # assign a random ID (or name it anything)

In [4]:
'''
Create the PersonGroup
'''
# Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
print('Person group:', PERSON_GROUP_ID)
face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

# Create Person
# Define woman friend
woman = face_client.person_group_person.create(PERSON_GROUP_ID, "Woman")
# Define man friend
man = face_client.person_group_person.create(PERSON_GROUP_ID, "Man")
# Define child friend
child = face_client.person_group_person.create(PERSON_GROUP_ID, "Child")

Person group: 1372d973-c5af-4e0b-9189-402bda027293


In [5]:
face_client.person_group.list()

In [6]:
face_client.person_group_person.list(PERSON_GROUP_ID)

In [7]:
'''
Detect faces and register to correct person
'''
# 学習用画像ファイルは"images/"に格納されているため移動
os.chdir('images/')
# Find all jpeg images of friends in working directory
woman_images = [file for file in glob.glob('*.jpg') if file.startswith("w")]
man_images = [file for file in glob.glob('*.jpg') if file.startswith("m")]
child_images = [file for file in glob.glob('*.jpg') if file.startswith("ch")]

# Add to a woman person
for image in woman_images:
    w = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, woman.person_id, w)

# Add to a man person
for image in man_images:
    m = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, man.person_id, m)

# Add to a child person
for image in child_images:
    ch = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, child.person_id, ch)

In [8]:
'''
Train PersonGroup
'''
print()
print('Training the person group...')
# Train the person group
face_client.person_group.train(PERSON_GROUP_ID)

while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print("Training status: {}.".format(training_status.status))
    print()
    print(training_status.status)

    # training_statusがsucceededになったら終了
    if (training_status.status is TrainingStatusType.succeeded):
        break
    # training_status==faildでメッセージを出して終了
    elif (training_status.status is TrainingStatusType.failed):
        face_client.person_group.delete(person_group_id=PERSON_GROUP_ID)
        sys.exit('Training the person group has failed.')
    time.sleep(5)


Training the person group...
Training status: running.

TrainingStatusType.running
Training status: succeeded.

TrainingStatusType.succeeded


In [9]:
os.getcwd()

'c:\\Users\\koishizu\\OneDrive - Microsoft\\日本マイクロソフト\\Demo_Material\\faceapi\\images'

In [10]:
'''
Identify a face against a defined PersonGroup
'''
# Group image for testing against
test_image_array = glob.glob('test-image-person-group.jpg')
image = open(test_image_array[0], 'r+b')

print('Pausing for 60 seconds to avoid triggering rate limit on free account...')
time.sleep (60)

# Detect faces
face_ids = []
# We use detection model 3 to get better performance.
# 画像に写っている顔の検出⇒複数個の可能性があるため配列
faces = face_client.face.detect_with_stream(image, detection_model='detection_03')
for face in faces:
    # facesとしてリストが返ってくるため、各インスタンスのface_idをリストに格納
    face_ids.append(face.face_id)

Pausing for 60 seconds to avoid triggering rate limit on free account...


In [24]:
# Identify faces
# テスト画像から検出したface_idのリストを用いて、特定グループ内に同一人物がいるか識別
# グループ指定にPERSON_GROUP_IDを利用
print(face_ids)
results = face_client.face.identify(face_ids, PERSON_GROUP_ID)
print(results)
print('Identifying faces in {}'.format(os.path.basename(image.name)))
if not results:
    print('No person identified in the person group for faces from {}.'.format(os.path.basename(image.name)))
for person in results:
    if len(person.candidates) > 0:
        print('Plausible person is {}.'.format(person.candidates[0].person_id))
        print(face_client.person_group_person.get(person_group_id=PERSON_GROUP_ID,person_id=person.candidates[0].person_id).name)
        print('Person for face ID {} is identified in {} with a confidence of {}.'.format(person.face_id, os.path.basename(image.name), person.candidates[0].confidence)) # Get topmost confidence score
    else:
        print('No person identified for face ID {} in {}.'.format(person.face_id, os.path.basename(image.name)))

['022d0d34-3e31-4b24-9e01-3851fb69fe97', 'f0c1a4ef-6112-4c35-a665-739158d43d93']
[<azure.cognitiveservices.vision.face.models._models_py3.IdentifyResult object at 0x000001EFBA074490>, <azure.cognitiveservices.vision.face.models._models_py3.IdentifyResult object at 0x000001EFBA074A00>]
Identifying faces in test-image-person-group.jpg
Plausible person is c1dde8f8-1cd6-4078-8ce6-ba222433e631.
Man
Person for face ID 022d0d34-3e31-4b24-9e01-3851fb69fe97 is identified in test-image-person-group.jpg with a confidence of 0.92235.
Plausible person is dcaa2f22-50de-4ad2-a851-30570bcdac1a.
Woman
Person for face ID f0c1a4ef-6112-4c35-a665-739158d43d93 is identified in test-image-person-group.jpg with a confidence of 0.93315.
